In [22]:
#Get historic 5 minute ticks of a stock in the last months

import ibapi
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates
import datetime
import matplotlib.dates as mdates
from datetime import date
import threading
import time
import pandas

class IBapi(EWrapper, EClient):
	def __init__(self):
		EClient.__init__(self, self)
		self.data = [] #Initialize variable to store candle

	def historicalData(self, reqId, bar):
		self.data.append([bar.date, bar.open, bar.close, bar.volume])
		print(bar)
		
def run_loop():
	app.run()

app = IBapi()
app.connect('127.0.0.1', 7496, 123)

#Start the socket in a thread
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()

time.sleep(1) #Sleep interval to allow time for connection to server

#Create contract object
contract = Contract()
contract.symbol = 'QQQ'
contract.secType = 'STK'
contract.exchange = 'SMART'
contract.currency = 'USD'

#Request historical candles
app.reqHistoricalData(1, contract, '', '6 M', '5 mins', 'TRADES', 1, 2, False, [])

time.sleep(20) #sleep to allow enough time for data to be returned

df = pandas.DataFrame(app.data, columns=['DateTime', 'Open', 'Close', 'Volume'])
df['DateTime'] = pandas.to_datetime(df['DateTime'],unit='s').dt.tz_localize(tz='UTC')
df['DateTime'] = df['DateTime'].dt.tz_convert(tz='US/Eastern')
df.to_csv('QqqHist.csv')  

print(df)


app.disconnect()

ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefnj


Empty DataFrame
Columns: [DateTime, Open, Close, Volume]
Index: []


In [ ]:
#Multiple Input Model

import ibapi
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, BatchNormalization
from tensorflow.keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler, EarlyStopping
import tensorflow as tf
from tensorflow import keras
import pandas
import datetime
from datetime import date
from sklearn.preprocessing import MinMaxScaler

df = pd.read_csv('C:\\src\\stockPredictor\\SpyHist.csv')

df['DateTime'] = pd.to_datetime(df['DateTime'])
df['date'] = [d.date() for d in df['DateTime']]
df['time'] = [d.time() for d in df['DateTime']]
df['Open'] = df['Open'].astype(float)
df['Close'] = df['Close'].astype(float)
df['Volume'] = df['Volume'].astype(int)
df['slowsma'] = df['Close'].rolling(21).mean()
df['fastsma'] = df['Close'].rolling(9).mean()
df['Difference'] = df['Close'] - df['Open']
df = df.dropna()

for currentDate in df.date.unique():
    if len(df.date.unique()) > len(df.loc[df['date'] == currentDate]):
        df = df.loc[df['date'] != currentDate]



scaler = MinMaxScaler()

df[['Difference', 'Volume', 'slowsma', 'fastsma']] = scaler.fit_transform(df[['Difference', 'Volume', 'slowsma', 'fastsma']])

maxSteps = len(df.time.unique())
count = 0

for i in range(1, maxSteps):
    # Create sequences of past and future data
    X = []
    y = []

    for currentDate in df.date.unique():
        past_sequence = df[['Difference','slowsma','fastsma','Volume']].loc[df['date']==currentDate][0:i]
        future_sequence = df[['Difference']][df['date']==currentDate][i:maxSteps]
        X.append(past_sequence)
        y.append(future_sequence)

    X = np.array(X)
    y = np.array(y)
    
    # Define the model
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(i, 4)))
    model.add(BatchNormalization())
    model.add(Dense(maxSteps - i))
    optimizer = Adam(clipnorm=0.001)
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    
    early_stopping = EarlyStopping(monitor='loss', patience=2, restore_best_weights=True)
    # Train the model (you can adjust the number of epochs and batch size)
    model.fit(X, y, epochs=100, callbacks=[early_stopping])

    modelName = 'model_'+str(df.time.unique()[i-1]).replace(':','_')+'.keras'
    print(modelName)
    model.save(modelName)

In [ ]:
#For fun, an average graph. averages all the differences of all the days and shows it in a graph. Objective is to show the average daily movement patterns

dfAverage = pandas.DataFrame(columns=['Average','Time'])
currentAve = 0
count = len(df.date.unique())

for currentTime in df.time.unique():
    average = 0
    for currentDate in df.date.unique():
        if(df.loc[(df['time'] == currentTime) & (df['date'] == currentDate)].empty is False):
            average = average + df.loc[(df['time'] == currentTime) & (df['date'] == currentDate)]['Close'].values[0]-df.loc[(df['time'] == currentTime) & (df['date'] == currentDate)]['Open'].values[0]
        else:
            print(currentDate)
    currentAve = currentAve + (average/count)
    dfAverage.loc[len(dfAverage.index)] = [currentAve,currentTime] 

datetimes = [datetime.datetime.combine(datetime.date.today(), t) for t in dfAverage["Time"]]
plt.plot(datetimes, dfAverage["Average"])
fmt = mdates.DateFormatter('%H:%M')
plt.gca().xaxis.set_major_formatter(fmt)


In [1]:
#build the models

from modelClasses import BasicModel, BatchClipEarlyStopD
import tableGetter

df = tableGetter.SpyHistory()
currentModel = BasicModel(df)
currentModel.build()